# `Olist Sales Analysis`

`Olist is a Brazilian e-commerce platform that connects small and medium-sized businesses to ``customers across Brazil. The platform operates as a marketplace, where merchants can list ``their products and services and customers can browse and purchase them online.`

`The Olist sales dataset available on Kaggle is a collection of anonymized data about orders placed on the Olist platform between January 2017 and August 2018. It contains a wide range of information about each order, including the order date, product details, payment and shipping information, customer and seller IDs, and customer reviews. The dataset also includes information about the sellers who list their products on Olist and data on customer behavior and demographics. The dataset is designed to help analysts and researchers better understand Brazil's e-commerce landscape and identify growth and optimization opportunities.  
`

### **Business Questions to help Olist gain better insights into their e-commerce platform and optimize available opportunities for growth**

## **1: What is the total revenue generated by Olist, and how has it changed over time?**

In [13]:
SELECT ROUND(SUM(op.payment_value),2) Total_Revenue
FROM [OlistBusiness].[dbo].[olist_orders_dataset] AS orders 
INNER JOIN [OlistBusiness].[dbo].[olist_order_payments_dataset] AS op
    ON (orders.order_id = op.order_id)
WHERE orders.order_status = 'delivered' OR orders.order_status = 'shipped';

SELECT YEAR(orders.order_purchase_timestamp) Years, 
       DATEPART(QUARTER, orders.order_purchase_timestamp) Quarters, ROUND(SUM(op.payment_value), 2) Total_Revenue
FROM [OlistBusiness].[dbo].[olist_orders_dataset] AS orders 
INNER JOIN [OlistBusiness].[dbo].[olist_order_payments_dataset] AS op
    ON (orders.order_id = op.order_id)
WHERE orders.order_status = 'delivered' OR orders.order_status = 'shipped'
GROUP BY YEAR(orders.order_purchase_timestamp), DATEPART(QUARTER, orders.order_purchase_timestamp)
ORDER BY Years, Quarters;

(1 row affected)

(9 rows affected)

Total execution time: 00:00:16.808

Total_Revenue
15599675.73


Years,Quarters,Total_Revenue
2016,3,136.23
2016,4,47833.08
2017,1,825050.51
2017,2,1469594.03
2017,3,1936082.71
2017,4,2770416.01
2018,1,3209937.08
2018,2,3310539.47
2018,3,2030086.61


## **2: How many orders were placed on Olist, and how does this vary by month or season?**

In [29]:
SELECT  COUNT(order_id) Total_Order_Placed FROM [OlistBusiness].[dbo].[olist_orders_dataset];

SELECT  MONTH(order_purchase_timestamp) Months, DATEPART(QUARTER, order_purchase_timestamp) Quarters,  COUNT(order_id) Total_Order_Placed  FROM [OlistBusiness].[dbo].[olist_orders_dataset]
GROUP BY MONTH(order_purchase_timestamp), DATEPART(QUARTER, order_purchase_timestamp)
ORDER BY Months;

(1 row affected)

(12 rows affected)

Total execution time: 00:00:00.148

Total_Order_Placed
99441


Months,Quarters,Total_Order_Placed
1,1,8069
2,1,8508
3,1,9893
4,2,9343
5,2,10573
6,2,9412
7,3,10318
8,3,10843
9,3,4305
10,4,4959


## **3: What are the most popular product categories on Olist, and how do their sales volumes compare to each other?**

In [3]:
SELECT TOP(10) pc.product_category_name_english, COUNT(ordersItems.order_id) Count_of_CAT, ROUND(SUM(ordersProd.payment_value), 2) SalesVolume  
FROM OlistBusiness.dbo.olist_order_payments_dataset ordersProd
JOIN OlistBusiness.dbo.olist_order_items_dataset ordersItems ON ordersProd.order_id = ordersItems.order_id
JOIN OlistBusiness.dbo.olist_products_dataset products ON ordersItems.product_id = products.product_id
JOIN OlistBusiness.dbo.product_category_name_translation pc ON pc.product_category_name = products.product_category_name
GROUP BY pc.product_category_name_english
ORDER BY SalesVolume DESC;

(10 rows affected)

Total execution time: 00:00:08.717

product_category_name_english,Count_of_CAT,SalesVolume
bed_bath_table,11823,1712553.67
health_beauty,9972,1657373.12
computers_accessories,8082,1585330.45
furniture_decor,8744,1430176.39
watches_gifts,6201,1429216.68
sports_leisure,8945,1392127.56
housewares,7355,1094758.13
auto,4379,852294.33
garden_tools,4574,838280.75
cool_stuff,3987,779698


## **4: What is the average order value (AOV) on Olist, and how does this vary by product category or payment method?**

In [21]:
SELECT 
        ROUND((Total_Revenue / Total_Order_Placed), 2) AS AOV
FROM (
        SELECT 
            SUM(op.payment_value) AS Total_Revenue
        FROM 
           [OlistBusiness].[dbo].[olist_orders_dataset] AS orders 
        INNER JOIN 
            [OlistBusiness].[dbo].[olist_order_payments_dataset] AS op
            ON (orders.order_id = op.order_id)
        WHERE 
            orders.order_status = 'delivered' OR orders.order_status = 'shipped'
    ) AS Revenue,
    (
        SELECT 
            COUNT(order_id) AS Total_Order_Placed
        FROM 
            [OlistBusiness].[dbo].[olist_orders_dataset]
    ) AS Orders;


SELECT payment_type, ROUND(SUM(payment_value)/COUNT(payment_value), 2) AOV
FROM OlistBusiness.dbo.olist_order_payments_dataset
GROUP BY payment_type
ORDER BY AOV DESC; 



(1 row affected)

(5 rows affected)

Total execution time: 00:00:08.036

AOV
156.87


payment_type,AOV
credit_card,163.32
boleto,145.03
debit_card,142.57
voucher,65.7
not_defined,0


## **5: How many sellers are active on Olist, and how does this number change over time?**

In [66]:
SELECT 
    YEAR(order_purchase_timestamp) AS year,
    MONTH(order_purchase_timestamp) AS month,
    COUNT(DISTINCT seller_id) AS active_sellers
FROM 
    OlistBusiness.dbo.olist_orders_dataset OO
INNER JOIN 
    OlistBusiness.dbo.olist_order_items_dataset OI ON OO.order_id = OI.order_id
WHERE 
    order_purchase_timestamp >= '2018-06-01' 
    AND order_purchase_timestamp <= '2018-12-31'
GROUP BY 
    YEAR(order_purchase_timestamp),
    MONTH(order_purchase_timestamp)
ORDER BY 
    year, 
    month;



(4 rows affected)

Total execution time: 00:00:03.251

year,month,active_sellers
2018,6,1175
2018,7,1261
2018,8,1278
2018,9,1


## **6: What is the distribution of seller ratings on Olist, and how does this impact sales performance?**

In [73]:
SELECT review_score, COUNT(DISTINCT OI.seller_id) RatingCount, ROUND(SUM(OOP.payment_value), 2)  Sales
FROM OlistBusiness.dbo.olist_order_items_dataset OI
JOIN OlistBusiness.dbo.olist_order_reviews_dataset OOR ON OI.order_id = OOR.order_id
JOIN OlistBusiness.dbo.olist_order_payments_dataset OOP ON OOR.order_id = OOP.order_id
GROUP BY review_score
ORDER BY RatingCount, Sales;

(5 rows affected)

Total execution time: 00:00:01.443

review_score,RatingCount,Sales
2,942,802158.87
3,1425,1668734.55
1,1797,3613316.25
4,1983,3659749.26
5,2700,10674329.22


## **7: How many customers have made repeat purchases on Olist, and what percentage of total sales do they account for?**

In [17]:
SELECT 
    COUNT(*) AS Repeated_Customers, ROUND(SUM(Sales), 2) TotalSalesCol,
    (ROUND((SUM(Sales)/(SELECT SUM(payment_value) PV FROM [OlistBusiness].[dbo].olist_order_payments_dataset)), 2) * 100) percentage_accounted
FROM(
    SELECT OC.customer_unique_id, COUNT(DISTINCT OO.order_id) Orders_count, SUM(OP.payment_value) Sales
    FROM [OlistBusiness].[dbo].[olist_customers_dataset] OC
    JOIN [OlistBusiness].[dbo].[olist_orders_dataset] OO ON OC.customer_id = OO.customer_id
    JOIN [OlistBusiness].[dbo].[olist_order_payments_dataset] OP ON OO.order_id = OP.order_id
    GROUP BY OC.customer_unique_id 
    HAVING COUNT(DISTINCT OO.order_id) > 1
) AS Total_Sales




(1 row affected)

Total execution time: 00:00:13.247

Repeated_Customers,TotalSalesCol,percentage_accounted
2997,944022.71,6


## **8: What is the average customer rating for products sold on Olist, and how does this impact sales performance?**

In [42]:
SELECT AVG(review_score)
FROM OlistBusiness.dbo.olist_order_reviews_dataset;



SELECT pc.product_category_name_english, AVG(OOR.review_score) Avg_Review_Count, ROUND(SUM(ordersProd.payment_value), 2) SalesVolume  
FROM OlistBusiness.dbo.olist_order_reviews_dataset OOR
JOIN OlistBusiness.dbo.olist_order_payments_dataset ordersProd ON OOR.order_id = ordersProd.order_id
JOIN OlistBusiness.dbo.olist_order_items_dataset ordersItems ON ordersProd.order_id = ordersItems.order_id
JOIN OlistBusiness.dbo.olist_products_dataset products ON ordersItems.product_id = products.product_id
JOIN OlistBusiness.dbo.product_category_name_translation pc ON pc.product_category_name = products.product_category_name
GROUP BY pc.product_category_name_english, OOR.review_score
ORDER BY SalesVolume DESC


(1 row affected)

(341 rows affected)

Total execution time: 00:00:01.923

(No column name)
4


product_category_name_english,Avg_Review_Count,SalesVolume
health_beauty,5,1003219.09
bed_bath_table,5,845149.07
watches_gifts,5,800919.42
sports_leisure,5,776243.77
computers_accessories,5,750479.32
furniture_decor,5,694359.88
housewares,5,578025.64
cool_stuff,5,455309.9
garden_tools,5,447460.12
auto,5,409194.63


## **9: What is the average order cancellation rate on Olist, and how does this impact seller performance?**

In [68]:
SELECT 
    Total_Order_canceled,
    Total_Order_Placed,
   ROUND(CAST(Total_Order_canceled AS FLOAT) / CAST(Total_Order_Placed AS FLOAT) * 100, 2)  AS Order_Cancel_Rate
FROM (
    SELECT 
        COUNT(*) AS Total_Order_canceled
    FROM 
       [OlistBusiness].[dbo].[olist_orders_dataset] AS orders 
    WHERE 
        orders.order_status = 'canceled'
) AS Revenue,
(
    SELECT 
        COUNT(order_id) AS Total_Order_Placed
    FROM 
        [OlistBusiness].[dbo].[olist_orders_dataset]
) AS Orders;


SELECT order_status, COUNT(order_status)
FROM [OlistBusiness].[dbo].[olist_orders_dataset]
GROUP BY order_status




(1 row affected)

(8 rows affected)

Total execution time: 00:00:00.482

Total_Order_canceled,Total_Order_Placed,Order_Cancel_Rate
625,99441,0.63


order_status,(No column name)
created,5
shipped,1107
canceled,625
approved,2
processing,301
unavailable,609
delivered,96478
invoiced,314


## **10: What are the top-selling products on Olist, and how have their sales trends changed over time?**

In [92]:
SELECT TOP(10) product_category_name_english, 
        COUNT(DISTINCT ordersItems.order_id) Count_of_CAT, 
        ROUND(SUM(payment_value), 2) SalesVolume,
        YEAR(order_purchase_timestamp), DATEPART(QUARTER, order_purchase_timestamp)

FROM [OlistBusiness].[dbo].[olist_order_payments_dataset] ordersProd
JOIN [OlistBusiness].[dbo].[olist_orders_dataset] orders ON ordersProd.order_id = orders.order_id
JOIN [OlistBusiness].[dbo].[olist_order_items_dataset] ordersItems ON ordersProd.order_id = ordersItems.order_id
JOIN [OlistBusiness].[dbo].[olist_products_dataset] products ON ordersItems.product_id = products.product_id
JOIN [OlistBusiness].[dbo].[product_category_name_translation] pc ON pc.product_category_name = products.product_category_name

GROUP BY product_category_name_english,
         YEAR(order_purchase_timestamp), 
         DATEPART(QUARTER, order_purchase_timestamp)
ORDER BY SalesVolume DESC;

(10 rows affected)

Total execution time: 00:00:07.008

product_category_name_english,Count_of_CAT,SalesVolume,(No column name),(No column name)
computers_accessories,2032,497936.57,2018,1
health_beauty,2111,403509.82,2018,2
watches_gifts,1517,368971.06,2018,2
bed_bath_table,1871,363745.34,2018,2
bed_bath_table,1962,351036.09,2018,1
sports_leisure,1843,350915.59,2018,1
health_beauty,1800,323460.71,2018,1
bed_bath_table,1743,319937.71,2017,4
health_beauty,1491,306633.56,2018,3
housewares,1383,292434.86,2018,2


## **11: Which payment methods are most commonly used by Olist customers, and how does this vary by product category or geographic region?**